# Juliet = RADAELOVE

In [1]:
import forge
from data import warehouse
from puzzle.puzzlepedia import prod_config
prod_config.init()

trie = warehouse.get('/words/unigram/trie')

In [50]:
import collections
from data.convert import str_to_str
from data.seek_sets import chain_seek_set

def parse(lines):
  lines = lines.strip().split('\n')
  tiles = []
  while lines:
    bottom = lines.pop()
    top = lines.pop()
    tiles.extend(tile for tile in zip(top.split(), bottom.split()))
  return tiles


_ORIGINAL = """
#  #  #  #  #  #  #  #  #
IT ON TW AS MO PH RO AL LO
A  A  C  C  D  E  E  E  E  E E  E  E
HA YK HE WA OT DI HT IO OV S TS VE WO
H  I  I  I  I  J  J  L  L  L  M  M  M
EE EL KE NT YO IA IN AT IT YT FL LF RD
M  M  N   N   N  N  N  N  O  O  O  O  P
SE TO DOM EIT LY PH RA UR EL FR PA US IG
P  P  R  R  S S  S  T  T  T  T  T  U
SC TY ON TH E ES RS AD DA OU TH UN CK
X  Y  Y  Y  ?  ?  ?  ?  ?  ?  ?  ?  ?
RE N  SE TI AN DO EN IC IL ME SE SI TL
"""

tiles = parse(_ORIGINAL)

bottoms = chain_seek_set.ChainSeekSet([bottom for _, bottom in tiles], 8)
_SETS = [bottom.lower() for _, bottom in tiles]
_CONVERSION_MAP = collections.defaultdict(list)
_CONVERSION_MAP[' '].append('')
_REVERSE_MAP = collections.defaultdict(list)

for top, bottom in tiles:
  _REVERSE_MAP[top.lower()].append(bottom.lower())
  if top == '#':
    continue
  _CONVERSION_MAP[bottom.lower()].append(top.lower())


In [35]:
for i, r in enumerate(str_to_str.str_to_str('#rth?indandfire', _REVERSE_MAP)):
  if i > 100:
    break
  print(r)

In [52]:
def walk(seek_set, acc, targets, pos=0):
  if pos >= len(targets):
    yield ' '.join(acc)
    return
  target = targets[pos]
  seek_set.set_length(target)
  for result, weight in trie.walk(seek_set, exact_match=True):
    if weight < 5e4:
      break
    acc.append(result)
    yield from walk(seek_set[result:], acc, targets, pos+1)
    acc.pop()

def process(top, bottom, targets):
  seek_set = chain_seek_set.ChainSeekSet(_SETS, sum(targets), prefix=bottom)
  found = []
  skipped = 0
  has_dot = False
  for result in walk(seek_set, [], targets):
    # on -> e
    for converted in str_to_str.str_to_str(result[2:], _CONVERSION_MAP):
      if converted.count('?') == 1:
        found.append(top + converted)
    if found:
      if has_dot:
        print()
        has_dot = False
      print(result, '=', ', '.join(found))
      found = []
    else:
      skipped += 1
      # print(result)
      if skipped % 100 == 0:
        print('.', end='')
        has_dot = True
      if skipped % 8000 == 0:
        print()
        has_dot = False
  print('skipped', skipped)

# R Eric Clapton: It's in the way that you use it (E?ic Clapton)
#   process('e', 'it', [3, 2, 3, 3, 4, 3, 3, 2])

# A Enya: Only Time
#   process('e', 'on', [4, 4])

# D Eddie Money: Two Tickets to Paradise (Ed?ie Money)
#   process('e', 'tw', [3, 7, 2, 8])

# A Etta James: A sunday kind of love (Ett? James)
#   process('e', 'as', [1, 6, 4, 2, 4])

# E Extreme: more than words (extr?me)
#   process('e', 'mo', [4, 4, 5])

# L Elton John: Philadelphia Freedom (e?tonjohn)
#   process('e', 'ph', [12, 7])

# O Europe: Rock the Night (eur?pe)
#   process('e', 'ro', [4, 3, 5])

# V Elvis Presley: A Little Less Conversation
#   process('e', 'al', [1, 6, 4, 12])

# E Eminem: Lose Yourself
#   process('e', 'lo', [4, 8])

a sunday kind of love
skipped 1


In [25]:
"""
2 12 3  3  4   3  3  2 | = 23
E R I C C L A P T O N
ITSINTHEWAYTHATYOUUSEIT
  R
  SI

4   4   | = 8
E N Y A
ONLYTIME
  ? = A
      ME

3  7      2 8       | = 20
E D D I E M O N E Y
TWOTICKETSTOPARADISE
    D
    IC

16     4   2 4   | = 17
E T T A J A M E S
ASUNDAYKINDOFLOVE
          A
          DO

4   4   5    | = 13
E X T R E M E
MORETHANWORDS
      R
      AN

12          7      | = 19
E ? T O N J O H N
PHILADELPHIAFREEDOM
  L
  IL

4   3  5    | = 12
E U R O P E
ROCKTHENIGHT
      O
      EN

16     4   12          | = 32
E L V I S P R E S L E Y
ALITTLELESSCONVERSATION
    V
    TL

4   8   .   | = 12
E M I N E M
LOSEYOURSELF
    ? = E
        SE

TOP [acehijlmnoprstuxy]
BOTTOM ^(ha|yk|he|wa|ot|di|ht|io|ov|s|ts|ve|wo|ee|el|ke|nt|yo|ia|in|at|it|yt|fl|lf|rd|se|to|dom|eit|ly|ph|ra|ur|el|fr|pa|us|ig|sc|ty|on|th|e|es|rs|ad|da|ou|th|un|ck|re|n|se|ti|an|do|en|ic|il|me|se|si|tl)$

2'1 2 3 3
4 3 3 2 4 4 3 7 2 8 1 6 4 2 4 4 4 5 12 7 4 3 5 1 6 4 12 4 8
= 147

top half
9 Es
18 double letters
=9 + 18 = 27

bottom half
65 single (top) letters
3 single (bottom) letters
6 triple letters (2x 3-letters)
120 double letters (65-3-2)*2
=65 + 3 + 6 + 120 = 194
OR
=18 (top) + 3 + 6 + 120 (bottom) = 147

"""

"\n2'1 2 3 3\n 4 3 3 2\n\ne.ic[acehijlmnoprstuxy]{6,8}\n2 12 3  3  4   3  3  2 | = 23\nE ? I C ? ? ? ? ? ? ?\nITSINTHE\nE ? I E\nITSINTWO\nE ? Y\nITSIN\n\n\n\nE E\nITS\nE P\nITSC\nE Y\nITSE\nE M\nITSE\nE ?\nITSE\nhe\ns\nin\nat\nit\nto\ndom\nus\non\nn\nan\ndo\nme\n\n\n4   4   | = 8\nE N O R\nONLYPATH\n\nE S  (esau esky esne espy esse esth)\nONES\n\nE N V Y\nONLYDOSE\n ? =V\n    DO\n\nE N V Y\nONLYANTI\n ? =V\n    AN\n\nE N D S\nONLYDOES\n ? =D\n    DO\n\nE M I T\nONTO....\n\nE O N S\nONUS \n\n\n\n^(an|do|en|ic|il|me|se|si|tl)(se|ti)$\n\n3  7      2 8       | = 20\nE R P E N ? ? ? ? ?\nTWONIGHTLY\nE R E I P\nTWONOVELTY\nE R E O P\nTWONOVELTY\nE D C L X\nTWOTHEATRE\nE D R P E\nTWOTONIGHT\nE D N J H\nTWOTRAINEE\nE E I E P\nTWOVELVETY\nE E O E P\nTWOVELVETY\nE E ? E O\nTWOVICIOUS\n^[tvnu](ha|yk|he|wa|ot|di|ht|io|ov|ts|ve|wo|ee|el|ke|nt|yo|ia|in|at|it|yt|fl|lf|rd|se|to|ly|ph|ra|ur|el|fr|pa|us|ig|sc|ty|on|th|es|rs|ad|da|ou|th|un|ck|re|se|ti|an|do|en|ic|il|me|se|si|tl){2,4}$\n^(.{1,6}|.{8,20})